In [ ]:
import logging
import asyncio
import pandas as pd
import polars as pl

import influxdb_client

from tastytrade.common.logging import setup_logging
from tastytrade.connections import Credentials, InfluxCredentials
from tastytrade.providers.market import MarketDataProvider
from tastytrade.providers.subscriptions import RedisSubscription
from tastytrade.config import RedisConfigManager

# Show all rows in pandas DataFrames
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

logging.getLogger().handlers.clear()

TEST = True
ENV = "Live"
DURATION = 15

EDT = 5

setup_logging(
    level=logging.DEBUG,
    log_dir="../logs",
    filename_prefix=f"{'dev' if TEST else 'prod'}_tastytrade",
    console=True,
    file=True,
)

loop = asyncio.get_event_loop()
loop.set_debug(True)
logging.getLogger("asyncio").setLevel(logging.DEBUG)

config = RedisConfigManager()
credentials = Credentials(config=config, env="Live")

influx_user = InfluxCredentials(config=config)
influxdb = influxdb_client.InfluxDBClient(
    url=influx_user.url, token=influx_user.token, org=influx_user.org
)

# Service Connections

In [ ]:
subscription = RedisSubscription(config=config)
await subscription.connect()

streamer = MarketDataProvider(subscription, influxdb)

In [ ]:
await streamer.subscribe("CandleEvent", "SPX{=*}")

In [ ]:
await streamer.unsubscribe("CandleEvent", "SPX{=*}")

In [ ]:
{event: streamer.frames[event].shape for event in streamer.frames.keys()}

In [ ]:
streamer["CandleEvent:SPX{=m}"].sort("time", descending=True)

In [ ]:
await streamer.subscribe("TradeEvent", "NVDA")

In [ ]:
streamer["TradeEvent:NVDA"].sort("time", descending=True)